# Context Engineering in LangChain 1.0 & LangGraph

Context engineering is about building **dynamic systems** that provide the right information and tools in the right format so that an LLM can accomplish its task. Think of the LLM as a CPU and its context window as RAM - we need to carefully curate what goes into that limited working memory.

This notebook covers the **LangChain 1.0+** patterns for context management:

1. **Static Context** - Immutable data that doesn't change during execution
2. **Dynamic Context** - Mutable data that evolves as the application runs  
3. **Runtime Context** - Data scoped to a single run using `context_schema`
4. **Cross-Conversation Context** - Data that persists across sessions using `InjectedStore`
5. **Unified Tool Access** - Using `InjectedState`, `InjectedStore`, and `ToolRuntime`

---

## Key LangChain 1.0+ Concepts

| Annotation | Purpose | Hidden from LLM? |
|------------|---------|------------------|
| `InjectedState` | Inject current graph state into tools | Yes |
| `InjectedStore` | Inject persistent memory store into tools | Yes |
| `ToolRuntime` | Unified access to state, store, context, config | Yes |
| `context_schema` | Define run-scoped context for agents | N/A |

---

## Setup

First, let's install the required packages and configure our environment.

In [ ]:
%pip install -qU langchain>=1.0.0 langchain-openai langchain-community langgraph>=0.4.0

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

---

# Part 1: Static Context

**Static context** is immutable data that doesn't change during execution. Examples include:

- User metadata (name, preferences set at startup)
- Database connections
- Available tools
- Configuration settings
- System prompts

In LangChain 1.0, static context is typically embedded in system prompts or accessed via closure.

## 1.1 Static Context via System Prompts

The simplest form of static context - information baked into the system prompt:

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

# Static context embedded in system prompt
STATIC_SYSTEM_PROMPT = """You are a customer support agent for TechCorp.

Company Information (STATIC CONTEXT):
- Company: TechCorp Inc.
- Support Hours: Mon-Fri 9am-5pm EST
- Refund Policy: 30-day money-back guarantee
- Current Promotion: 20% off with code SAVE20

Use this information to help customers with their inquiries.
"""

model = ChatOpenAI(model="gpt-4o-mini")

messages = [
    SystemMessage(content=STATIC_SYSTEM_PROMPT),
    HumanMessage(content="What's your refund policy?")
]

response = model.invoke(messages)
print(response.content)

At TechCorp Inc., we offer a 30-day money-back guarantee. If you are not satisfied with your purchase, you can request a refund within 30 days of your purchase date. If you have any specific questions or need further assistance with the refund process, feel free to ask!


## 1.2 Static Context via Closure (Module-Level Config)

A simple pattern: define configuration once, and tools/functions access it via closure:

In [3]:
# Static configuration - defined once, never changes during execution
PRODUCT_CATALOG = {
    "Widget Pro": {"price": 99.99, "in_stock": True},
    "Gadget Plus": {"price": 149.99, "in_stock": True},
    "Device Max": {"price": 299.99, "in_stock": False},
}

MAX_DISCOUNT_PERCENT = 25
SUPPORT_EMAIL = "support@techcorp.com"

from langchain_core.tools import tool

@tool
def check_product(product_name: str) -> str:
    """Check if a product exists and its availability."""
    # Tools access static context via closure
    if product_name in PRODUCT_CATALOG:
        product = PRODUCT_CATALOG[product_name]
        status = "in stock" if product["in_stock"] else "out of stock"
        return f"{product_name}: ${product['price']} ({status})"
    return f"Product '{product_name}' not found. Available: {list(PRODUCT_CATALOG.keys())}"

@tool
def calculate_price(product_name: str, discount_percent: int = 0) -> str:
    """Calculate final price with optional discount."""
    if product_name not in PRODUCT_CATALOG:
        return f"Product '{product_name}' not found."
    
    base_price = PRODUCT_CATALOG[product_name]["price"]
    # Static context (MAX_DISCOUNT_PERCENT) limits the discount
    actual_discount = min(discount_percent, MAX_DISCOUNT_PERCENT)
    final_price = base_price * (1 - actual_discount / 100)
    
    result = f"{product_name}: ${final_price:.2f}"
    if discount_percent > MAX_DISCOUNT_PERCENT:
        result += f" (discount capped at {MAX_DISCOUNT_PERCENT}%)"
    return result

# Test the tools - they use static context
print(check_product.invoke({"product_name": "Widget Pro"}))
print(check_product.invoke({"product_name": "Unknown Product"}))
print(calculate_price.invoke({"product_name": "Widget Pro", "discount_percent": 50}))

Widget Pro: $99.99 (in stock)
Product 'Unknown Product' not found. Available: ['Widget Pro', 'Gadget Plus', 'Device Max']
Widget Pro: $74.99 (discount capped at 25%)


## 1.3 Static Context with ReAct Agent

Let's build a simple agent that uses these tools with static context:

In [5]:
from langchain.agents import create_agent

# Create a ReAct agent with our tools
# The tools have access to static context (PRODUCT_CATALOG, MAX_DISCOUNT_PERCENT)
shop_assistant = create_agent(
    model="openai:gpt-5-mini",
    tools=[check_product, calculate_price],
    system_prompt="You are a helpful shop assistant. Use tools to help customers with product info and pricing."
)

In [6]:
# The agent uses tools that access static context
result = shop_assistant.invoke({
    "messages": [{"role": "user", "content": "What's the price of Widget Pro with a 20% discount?"}]
})

for msg in result["messages"]:
    if hasattr(msg, 'pretty_print'):
        msg.pretty_print()
    else:
        print(f"{msg['role']}: {msg['content']}")

================================ Human Message =================================

What's the price of Widget Pro with a 20% discount?
================================== Ai Message ==================================
Tool Calls:
  calculate_price (call_6BFGcFzRIaNblb07KFj9YIVK)
 Call ID: call_6BFGcFzRIaNblb07KFj9YIVK
  Args:
    product_name: Widget Pro
    discount_percent: 20
================================= Tool Message =================================
Name: calculate_price

Widget Pro: $79.99
================================== Ai Message ==================================

With a 20% discount, the price of the Widget Pro is $79.99.


In [7]:
# Another query - notice the static context (product catalog) is always available
result = shop_assistant.invoke({
    "messages": [{"role": "user", "content": "Is Device Max available? And what products do you have?"}]
})

print(result["messages"][-1].content)

Device Max is currently out of stock; its price is $299.99.

Would you like me to:
- Show all available products and prices now,
- Look for similar/alternative models (e.g., Device Pro, Device Mini), or
- Set a restock notification for Device Max?

Tell me which option (or any filters like price range, features, or category) and I’ll fetch the list.


---

# Part 2: Dynamic Context with InjectedState

**Dynamic context** is mutable data that evolves as the application runs. In LangChain 1.0+, we use `InjectedState` to give tools access to the current graph state without exposing it to the LLM.

**Key Benefits:**
- Tools can read/write state without the LLM seeing implementation details
- Reduces prompt size (20-30% reduction in routing latency)
- Enhanced tool selection stability

## 2.1 Basic InjectedState Pattern

The `InjectedState` annotation marks parameters that should be injected automatically, hidden from the LLM's view:

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.prebuilt import create_react_agent, InjectedState
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# Define our state with dynamic context
class ShoppingState(TypedDict):
    messages: Annotated[list, add_messages]
    cart: list[str]  # Dynamic context: items in cart
    total_items: int  # Dynamic context: computed value


@tool
def add_to_cart(
    item: str,
    state: Annotated[dict, InjectedState]  # Injected - not visible to LLM
) -> str:
    """Add an item to the shopping cart."""
    # Access dynamic context from state
    current_cart = state.get("cart", [])
    new_cart = current_cart + [item]
    return f"Added '{item}' to cart. Cart now has {len(new_cart)} items: {new_cart}"


@tool
def view_cart(
    state: Annotated[dict, InjectedState]  # Injected - not visible to LLM
) -> str:
    """View the current shopping cart contents."""
    cart = state.get("cart", [])
    if not cart:
        return "Your cart is empty."
    return f"Cart contents ({len(cart)} items): {', '.join(cart)}"


# Notice: The LLM only sees "item" parameter for add_to_cart,
# and no parameters for view_cart. The state injection is hidden.
print("add_to_cart schema (LLM view):")
print(add_to_cart.get_input_schema().model_json_schema())

## 2.2 Injecting Specific State Fields

Instead of injecting the entire state, you can inject specific fields:

In [9]:
@tool
def get_cart_summary(
    cart: Annotated[list, InjectedState("cart")]  # Inject only the 'cart' field
) -> str:
    """Get a summary of the shopping cart."""
    if not cart:
        return "Cart is empty."
    from collections import Counter
    counts = Counter(cart)
    summary = [f"{item}: {count}" for item, count in counts.items()]
    return f"Cart summary: {', '.join(summary)}"


@tool
def check_cart_size(
    total_items: Annotated[int, InjectedState("total_items")]  # Inject specific field
) -> str:
    """Check if the cart has items."""
    if total_items == 0:
        return "Your cart is empty. Add some items!"
    return f"You have {total_items} item(s) in your cart."


# Both tools have no visible parameters to the LLM
print("get_cart_summary schema (LLM view):")
print(get_cart_summary.get_input_schema().model_json_schema())

get_cart_summary schema (LLM view):
{'description': 'Get a summary of the shopping cart.', 'properties': {'cart': {'items': {}, 'title': 'Cart', 'type': 'array'}}, 'required': ['cart'], 'title': 'get_cart_summary', 'type': 'object'}


## 2.3 Building a Stateful Agent with InjectedState

Let's create a complete agent that maintains dynamic state:

In [11]:
from langgraph.prebuilt import create_react_agent, InjectedState, ToolNode
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from typing import Annotated, Literal
from typing_extensions import TypedDict

# State with dynamic context
class CartState(TypedDict):
    messages: Annotated[list, add_messages]
    cart: list[str]


@tool
def add_item(
    item: str,
    quantity: int = 1,
    state: Annotated[dict, InjectedState] = None
) -> str:
    """Add item(s) to the cart.
    
    Args:
        item: Name of the item to add
        quantity: How many to add (default 1)
    """
    current_cart = state.get("cart", []) if state else []
    items_to_add = [item] * quantity
    new_cart = current_cart + items_to_add
    return f"Added {quantity}x {item}. Cart: {new_cart}"


@tool  
def show_cart(
    state: Annotated[dict, InjectedState] = None
) -> str:
    """Display current cart contents."""
    cart = state.get("cart", []) if state else []
    if not cart:
        return "Cart is empty."
    from collections import Counter
    counts = Counter(cart)
    items = [f"{item} x{count}" for item, count in counts.items()]
    return f"Cart ({len(cart)} total): {', '.join(items)}"


@tool
def clear_cart(
    state: Annotated[dict, InjectedState] = None
) -> str:
    """Clear all items from the cart."""
    return "Cart cleared!"


model = ChatOpenAI(model="gpt-4o-mini")
tools = [add_item, show_cart, clear_cart]

# Create agent with state schema
cart_agent = create_agent(
    model=model,
    tools=tools,
    system_prompt="You are a shopping assistant. Help users manage their cart.",
    state_schema=CartState
)

In [12]:
# Test the stateful agent
result = cart_agent.invoke({
    "messages": [{"role": "user", "content": "Add 2 apples and 1 banana to my cart"}],
    "cart": []  # Initialize empty cart
})

print("Final response:")
print(result["messages"][-1].content)

Final response:
I've added 2 apples and 1 banana to your cart. If you need anything else, feel free to ask!


---

# Part 3: Runtime Context with context_schema

**Runtime context** is data scoped to a single run or invocation. In LangChain 1.0+, use `context_schema` to define immutable per-run data like user IDs, database handles, or session configuration.

**Key difference from state:**
- State is mutable and flows through the graph
- Context is immutable configuration for this specific run

## 3.1 Defining Runtime Context Schema

Use a dataclass to define the context your agent needs:

In [ ]:
from dataclasses import dataclass
from langchain.agents import create_agent
from langgraph.config import get_config
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.runnables import RunnableConfig
from typing import Annotated
from langchain_core.tools import InjectedToolArg

@dataclass
class UserContext:
    """Runtime context - immutable data for this specific run."""
    user_id: str
    user_tier: str  # "free", "premium", "enterprise"
    language: str = "en"


@tool
def get_personalized_greeting(
    time_of_day: str,
    config: Annotated[RunnableConfig, InjectedToolArg]
) -> str:
    """Get a personalized greeting based on user context.
    
    Args:
        time_of_day: morning, afternoon, or evening
    """
    # Access runtime context from config
    ctx = config.get("configurable", {})
    user_id = ctx.get("user_id", "guest")
    user_tier = ctx.get("user_tier", "free")
    language = ctx.get("language", "en")
    
    greetings = {
        "en": {"morning": "Good morning", "afternoon": "Good afternoon", "evening": "Good evening"},
        "es": {"morning": "Buenos dias", "afternoon": "Buenas tardes", "evening": "Buenas noches"},
        "ja": {"morning": "Ohayo gozaimasu", "afternoon": "Konnichiwa", "evening": "Konbanwa"},
    }
    
    greeting = greetings.get(language, greetings["en"]).get(time_of_day, "Hello")
    tier_suffix = " (Premium Member)" if user_tier == "premium" else ""
    
    return f"{greeting}, User {user_id}{tier_suffix}!"


@tool
def check_feature_access(
    feature: str,
    config: Annotated[RunnableConfig, InjectedToolArg]
) -> str:
    """Check if user has access to a feature based on their tier.
    
    Args:
        feature: Name of the feature to check
    """
    ctx = config.get("configurable", {})
    user_tier = ctx.get("user_tier", "free")
    
    # Feature access by tier
    tier_features = {
        "free": ["basic_search", "view_products"],
        "premium": ["basic_search", "view_products", "advanced_search", "priority_support"],
        "enterprise": ["basic_search", "view_products", "advanced_search", "priority_support", "api_access", "bulk_export"]
    }
    
    available = tier_features.get(user_tier, [])
    if feature in available:
        return f"Access granted to '{feature}' for {user_tier} tier."
    return f"Access denied. '{feature}' requires upgrade from {user_tier} tier."


# Test runtime context injection
print("English premium user:")
print(get_personalized_greeting.invoke(
    {"time_of_day": "morning"},
    config={"configurable": {"user_id": "alice123", "user_tier": "premium", "language": "en"}}
))

print("\nJapanese free user:")
print(get_personalized_greeting.invoke(
    {"time_of_day": "morning"},
    config={"configurable": {"user_id": "bob456", "user_tier": "free", "language": "ja"}}
))

## 3.2 Using context_schema with Agents

The `context_schema` parameter lets you define runtime context for your agent:

In [ ]:
from dataclasses import dataclass
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

@dataclass
class AppContext:
    """Application runtime context."""
    user_id: str
    session_id: str
    user_tier: str = "free"


model = ChatOpenAI(model="gpt-4o-mini")

# Create agent with context schema
context_agent = create_react_agent(
    model=model,
    tools=[get_personalized_greeting, check_feature_access],
    prompt="You are a helpful assistant. Use tools to personalize the experience.",
)

In [ ]:
# Run with different runtime contexts
print("=== Premium User Session ===")
result = context_agent.invoke(
    {"messages": [{"role": "user", "content": "Greet me! It's morning. Also check if I can use api_access."}]},
    config={"configurable": {"user_id": "premium_alice", "session_id": "sess_001", "user_tier": "premium"}}
)
print(result["messages"][-1].content)

In [ ]:
print("\n=== Free User Session ===")
result = context_agent.invoke(
    {"messages": [{"role": "user", "content": "Greet me! It's afternoon. Can I use api_access?"}]},
    config={"configurable": {"user_id": "free_bob", "session_id": "sess_002", "user_tier": "free"}}
)
print(result["messages"][-1].content)

---

# Part 4: Cross-Conversation Context with InjectedStore

**Cross-conversation context** persists across multiple sessions. In LangChain 1.0+, use `InjectedStore` to give tools access to persistent storage.

**Key Benefits:**
- Long-term memory across conversations
- User preferences, facts, and history
- Hidden from LLM's tool schema

## 4.1 Basic InjectedStore Pattern

The `InjectedStore` annotation provides tools with access to the persistent store:

In [ ]:
from typing import Annotated, Any
from langgraph.prebuilt import InjectedStore
from langgraph.store.memory import InMemoryStore
from langchain_core.tools import tool

@tool
def remember_user_fact(
    user_id: str,
    fact_type: str,
    fact_value: str,
    store: Annotated[Any, InjectedStore()]  # Injected - hidden from LLM
) -> str:
    """Remember a fact about a user for future conversations.
    
    Args:
        user_id: The user's identifier
        fact_type: Category like 'name', 'preference', 'hobby'
        fact_value: The actual fact to remember
    """
    namespace = ("users", user_id, "facts")
    store.put(namespace, fact_type, {"value": fact_value})
    return f"Remembered: {user_id}'s {fact_type} is '{fact_value}'"


@tool
def recall_user_facts(
    user_id: str,
    store: Annotated[Any, InjectedStore()]  # Injected - hidden from LLM
) -> str:
    """Recall all stored facts about a user.
    
    Args:
        user_id: The user's identifier
    """
    namespace = ("users", user_id, "facts")
    items = store.search(namespace)
    
    if not items:
        return f"No memories stored for user '{user_id}'"
    
    facts = [f"- {item.key}: {item.value['value']}" for item in items]
    return f"Memories for {user_id}:\n" + "\n".join(facts)


@tool
def get_specific_fact(
    user_id: str,
    fact_type: str,
    store: Annotated[Any, InjectedStore()]  # Injected - hidden from LLM
) -> str:
    """Get a specific fact about a user.
    
    Args:
        user_id: The user's identifier
        fact_type: The type of fact to retrieve
    """
    namespace = ("users", user_id, "facts")
    item = store.get(namespace, fact_type)
    
    if item is None:
        return f"No '{fact_type}' stored for {user_id}"
    return f"{user_id}'s {fact_type}: {item.value['value']}"


# Check that store is hidden from LLM
print("remember_user_fact schema (LLM sees):")
print(remember_user_fact.get_input_schema().model_json_schema())

## 4.2 Building a Memory-Enabled Agent

Create an agent with persistent cross-conversation memory:

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore
from langchain_openai import ChatOpenAI

# Create persistent store
memory_store = InMemoryStore()

model = ChatOpenAI(model="gpt-4o-mini")

# Create agent with store for cross-conversation memory
memory_agent = create_react_agent(
    model=model,
    tools=[remember_user_fact, recall_user_facts, get_specific_fact],
    prompt="""You are a helpful assistant with long-term memory.
When users share personal information, use remember_user_fact to store it.
When users ask what you know about them, use recall_user_facts.
Always use the user_id from the conversation.""",
    store=memory_store  # Inject the store
)

In [ ]:
# Session 1: User shares information
print("=== Session 1: Alice introduces herself ===")
result = memory_agent.invoke({
    "messages": [{
        "role": "user", 
        "content": "Hi! I'm user alice_123. My name is Alice, I work as a data scientist, and I love hiking."
    }]
})
print(f"Agent: {result['messages'][-1].content}")

# Check what was stored
print("\nStored in memory store:")
for item in memory_store.search(("users", "alice_123", "facts")):
    print(f"  {item.key}: {item.value}")

In [ ]:
# Session 2: NEW conversation thread - but memories persist!
print("=== Session 2: New conversation, same user ===")
result = memory_agent.invoke({
    "messages": [{
        "role": "user", 
        "content": "I'm alice_123. What do you remember about me?"
    }]
})
print(f"Agent: {result['messages'][-1].content}")

In [ ]:
# Session 3: Different user - isolated memories
print("=== Session 3: Different user (bob_456) ===")
result = memory_agent.invoke({
    "messages": [{
        "role": "user", 
        "content": "I'm bob_456. What do you know about me?"
    }]
})
print(f"Agent: {result['messages'][-1].content}")

## 4.3 Thread-Scoped vs Cross-Conversation Memory

Combine `MemorySaver` (thread-scoped) with `InMemoryStore` (cross-conversation):

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

# Thread-scoped: conversation history within a session
checkpointer = MemorySaver()

# Cross-conversation: long-term user facts
long_term_store = InMemoryStore()

model = ChatOpenAI(model="gpt-4o-mini")

# Agent with both memory types
full_memory_agent = create_react_agent(
    model=model,
    tools=[remember_user_fact, recall_user_facts],
    prompt="You are a helpful assistant with both conversation history and long-term memory.",
    checkpointer=checkpointer,  # Thread-scoped (conversation history)
    store=long_term_store       # Cross-conversation (user facts)
)

In [ ]:
# Thread 1: Build conversation history AND store facts
thread_config = {"configurable": {"thread_id": "conversation_001"}}

print("=== Thread 1, Turn 1 ===")
result = full_memory_agent.invoke(
    {"messages": [{"role": "user", "content": "Hi, I'm charlie_789. My favorite color is blue."}]},
    config=thread_config
)
print(f"Agent: {result['messages'][-1].content}")

print("\n=== Thread 1, Turn 2 ===")
result = full_memory_agent.invoke(
    {"messages": [{"role": "user", "content": "What was my favorite color again?"}]},
    config=thread_config
)
print(f"Agent: {result['messages'][-1].content}")

In [ ]:
# Thread 2: NEW conversation - no history, but long-term memory persists
new_thread_config = {"configurable": {"thread_id": "conversation_002"}}

print("=== Thread 2 (NEW conversation) ===")
result = full_memory_agent.invoke(
    {"messages": [{"role": "user", "content": "I'm charlie_789. Do you remember my favorite color?"}]},
    config=new_thread_config
)
print(f"Agent: {result['messages'][-1].content}")
print("\n(Note: Agent uses recall_user_facts to access long-term memory, not conversation history)")

---

# Part 5: Scratchpad Pattern (Working Memory)

A **scratchpad** is dynamic context where the agent writes notes to itself during execution. This is useful for:

- Tracking intermediate results
- Building up analysis over multiple steps
- Maintaining focus on multi-step tasks

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated, List
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

class ScratchpadState(TypedDict):
    """State with a scratchpad for dynamic notes."""
    messages: Annotated[list, add_messages]
    scratchpad: List[str]  # Dynamic context: notes the agent writes
    step_count: int        # Dynamic context: tracks progress


model = ChatOpenAI(model="gpt-4o-mini")


def analyze_and_note(state: ScratchpadState):
    """Agent analyzes input and writes to scratchpad."""
    user_msg = state["messages"][-1].content
    current_notes = state.get("scratchpad", [])
    step = state.get("step_count", 0) + 1
    
    # Build prompt with current scratchpad context
    scratchpad_context = "\n".join(current_notes) if current_notes else "(empty)"
    
    prompt = f"""Analyze this request and write a brief note about what you learned.

Current scratchpad:
{scratchpad_context}

User request: {user_msg}

Respond with:
1. A note to add to your scratchpad (start with "NOTE:")
2. Your response to the user (start with "RESPONSE:")"""
    
    response = model.invoke([HumanMessage(content=prompt)])
    content = response.content
    
    # Extract note and response
    note = ""
    reply = content
    if "NOTE:" in content and "RESPONSE:" in content:
        parts = content.split("RESPONSE:")
        note = parts[0].replace("NOTE:", "").strip()
        reply = parts[1].strip()
    
    # Update scratchpad (dynamic context)
    new_notes = current_notes + [f"Step {step}: {note}"] if note else current_notes
    
    return {
        "messages": [response],
        "scratchpad": new_notes,
        "step_count": step
    }


# Build graph
builder = StateGraph(ScratchpadState)
builder.add_node("analyze", analyze_and_note)
builder.add_edge(START, "analyze")
builder.add_edge("analyze", END)

scratchpad_graph = builder.compile()

In [ ]:
# Run multiple interactions, watching scratchpad grow
state = {
    "messages": [HumanMessage(content="I need help planning a trip to Japan")],
    "scratchpad": [],
    "step_count": 0
}

result = scratchpad_graph.invoke(state)
print("After turn 1:")
print(f"Scratchpad: {result['scratchpad']}")
print(f"Step count: {result['step_count']}")

# Continue conversation - scratchpad accumulates
result["messages"].append(HumanMessage(content="I'm interested in traditional culture and food"))
result = scratchpad_graph.invoke(result)
print("\nAfter turn 2:")
print(f"Scratchpad: {result['scratchpad']}")
print(f"Step count: {result['step_count']}")

---

# Summary: Context Types in LangChain 1.0+

| Context Type | Mutability | Lifetime | LangChain 1.0 Mechanism | Example |
|-------------|------------|----------|------------------------|--------|
| **Static** | Immutable | Application lifetime | System prompts, closure | Product catalog, API limits |
| **Dynamic** | Mutable | Within a run | `InjectedState`, StateGraph | Shopping cart, scratchpad |
| **Runtime** | Immutable per-run | Single invocation | `context_schema`, `RunnableConfig` | User ID, tier, language |
| **Cross-Conversation** | Persistent | Multiple sessions | `InjectedStore`, `InMemoryStore` | User facts, preferences |

---

## Key LangChain 1.0+ Patterns

### 1. InjectedState - Access graph state in tools
```python
@tool
def my_tool(query: str, state: Annotated[dict, InjectedState]) -> str:
    # state is injected, hidden from LLM
    cart = state.get("cart", [])
    ...
```

### 2. InjectedStore - Access persistent storage in tools
```python
@tool
def save_fact(key: str, value: str, store: Annotated[Any, InjectedStore()]) -> str:
    # store is injected, hidden from LLM
    store.put(("facts",), key, value)
    ...
```

### 3. context_schema - Define run-scoped context
```python
@dataclass
class AppContext:
    user_id: str
    tier: str

agent = create_react_agent(..., context_schema=AppContext)
agent.invoke({...}, context=AppContext(user_id="alice", tier="premium"))
```

---

## Four Strategies for Context Management

1. **Write** - Save context externally (scratchpad, store)
2. **Select** - Pull relevant context in (search, filtering)
3. **Compress** - Summarize to reduce tokens
4. **Isolate** - Separate contexts across agents/fields

---

**Resources:**
- [LangChain Tools Documentation](https://docs.langchain.com/oss/python/langchain/tools)
- [LangGraph Agents Reference](https://reference.langchain.com/python/langgraph/agents/)
- [Context Engineering Repository](https://github.com/langchain-ai/context_engineering)